# Label Selection

This is a **manual workflow**. Once a dataset has been populated with the results from the **Ensemble Exploration** workflow, this workflow can be used to select the samples to be labeled. Here you manually select the final images to be labeled. These will be exported and are ready to be labeled.


In [31]:
from fiftyone import ViewField as F
import fiftyone as fo
import os

In [26]:
# Loading dataset and filtering view for optimized performance
# Shows only samples with min. n unique detections from Ensemble Exploration
# Shows only labels that belong to the unique detections
dataset_name = "mcity_fisheye_3_months" # Dataset to load
dataset = fo.load_dataset(dataset_name)
min_n_unique_exploration = 3
view_n_unique_exploration = dataset.match(F("n_unique_exploration") >= min_n_unique_exploration)
view_tagged_labels = view_n_unique_exploration.select_labels(tags="vru_overlap")    # FIXME detections_overlap

In [27]:
# Use this view to label the samples you want to have labeled
# Choose the tag_for_labeling to tag the samples you want to label
tag_for_labeling = "select_vru" # FIXME todo_label
fo.launch_app(view=view_tagged_labels)

Dataset:          mcity_fisheye_3_months
Media type:       image
Num samples:      10653
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/
View stages:
    1. Match(filter={'$expr': {'$gte': [...]}})
    2. SelectLabels(labels=None, ids=None, tags=['vru_overlap'], fields=None, omit_empty=True)

In [28]:
# Inspect your selection of samples to be labeled
view_selection = dataset.match_tags(tag_for_labeling)
print(view_selection)

Dataset:     mcity_fisheye_3_months
Media type:  image
Num samples: 100
Sample fields:
    id:                                           fiftyone.core.fields.ObjectIdField
    filepath:                                     fiftyone.core.fields.StringField
    tags:                                         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:                                   fiftyone.core.fields.DateTimeField
    last_modified_at:                             fiftyone.core.fields.DateTimeField
    ground_truth:                                 fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    location:                                     fiftyone.core.fields.StringField
    name:                                         fiftyone.core.fields.StringField
    timestamp:                       

In [29]:
# Export the dataset you want to have labeled
# This example uses the YOLOv5Dataset format which also exports labels
# https://docs.voxel51.com/user_guide/export_datasets.html#yolov5dataset

splits = ["train", "val"]                                                   # Select all the splits your dataset has
export_dir="/media/dbogdoll/Datasets/vru_labeling"                          # Directory to export the files to
label_field="pred_google_owlv2_large_patch14_finetuned"                     # Include a prediction field from one of the zero-shot models

classes=view_selection.distinct(f"{label_field}.detections.label")          # Get all the classes from the prediction field

# Export the splits
for split in splits:
    split_view = view_selection.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split=split,
        classes=classes,
    )

Directory '/media/dbogdoll/Datasets/vru_labeling' already exists; export will be merged with existing files
 100% |███████████████████| 96/96 [5.4s elapsed, 0s remaining, 24.6 samples/s]      
Directory '/media/dbogdoll/Datasets/vru_labeling' already exists; export will be merged with existing files
 100% |█████████████████████| 4/4 [108.1ms elapsed, 0s remaining, 37.0 samples/s] 


In [ ]:
# If you want to label the dataset in-house, this example shows how to use Label Studio. Voxel51 also has other integrations
# https://docs.voxel51.com/integrations/labelstudio.html
# To setup Label Studio, follow the instructions here: https://docs.voxel51.com/integrations/labelstudio.html#label-studio-setup

classes_for_labeling = ["car", "truck", "bus", "trailer", "motorbike/cycler", "pedestrian", "van", "pickup"]    # Classes from Mcity Fisheye dataset
label_schema = {
    "new_ground_truth": {
        "type": "detections",
        "classes": classes_for_labeling,
    },
}

# Connect to Label Studio
label_studio_api_key = os.environ.get("FIFTYONE_LABELSTUDIO_API_KEY")
with open(".secret", "r") as file:
    for line in file:
        key, value = line.strip().split("=")
        if key == "FIFTYONE_LABELSTUDIO_API_KEY":
            label_studio_api_key = value

anno_key = "labelstudio"
view_selection.annotate(
    anno_key,
    backend="labelstudio",
    label_schema=label_schema,
    launch_editor=True,
    api_key=label_studio_api_key
)

The backend 'labelstudio' does not support attributes. Provided attributes will be ignored.
Please enter your API key.
You can avoid this in the future by setting your `FIFTYONE_LABELSTUDIO_API_KEY` environment variable


HTTPError: 404 Client Error: Not Found for url: https://labelstud.io/user/login